In [4]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 15
year = 2022

data = get_data(day=day, year=year)



In [2]:
# data = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
# Sensor at x=9, y=16: closest beacon is at x=10, y=16
# Sensor at x=13, y=2: closest beacon is at x=15, y=3
# Sensor at x=12, y=14: closest beacon is at x=10, y=16
# Sensor at x=10, y=20: closest beacon is at x=10, y=16
# Sensor at x=14, y=17: closest beacon is at x=10, y=16
# Sensor at x=8, y=7: closest beacon is at x=2, y=10
# Sensor at x=2, y=0: closest beacon is at x=2, y=10
# Sensor at x=0, y=11: closest beacon is at x=2, y=10
# Sensor at x=20, y=14: closest beacon is at x=25, y=17
# Sensor at x=17, y=20: closest beacon is at x=21, y=22
# Sensor at x=16, y=7: closest beacon is at x=15, y=3
# Sensor at x=14, y=3: closest beacon is at x=15, y=3
# Sensor at x=20, y=1: closest beacon is at x=15, y=3"""

In [31]:
coords = [[int(i) for i in re.match(pattern=r"[^=]+=(\-?\d+), y=(\-?\d+)[^=]+=(\-?\d+), y=(\-?\d+)", 
          string=l).groups()] for l in data.split('\n')]

manhattan = lambda x1,y1,x2,y2: abs(x1-x2)+abs(y1-y2)
sensors = [(c[0],c[1], manhattan(*c)) for c in coords]

In [7]:
from tqdm.auto import tqdm

y = 2000000
part1 = sum(1 for x in tqdm(range(min(sx-m+sy-y for sx,sy,m in sensors), max(sx+m-sy+y for sx,sy,m in sensors))) 
       if not any(((x,y) == (bx, by)) or ((x,y) == (sx, sy)) for sx,sy,bx,by in coords) 
       and any(manhattan(x,y,sx,sy) <= m for sx,sy,m in sensors))

print("Part 1: ", part1)

max_c = 4000000

found = False 
for y in tqdm(range(0, max_c+1)):
    x = 0
    while (x <= max_c) and not found:
        dm = max(m - manhattan(x,y,sx,sy) for sx,sy,m in sensors)
        if dm < 0:
            found = True
            print('Part 2:', x*4000000+y)
            break
        x += max(1, dm)
    if found:
        break

100%|██████████| 9011697/9011697 [00:50<00:00, 177227.45it/s]


5607466

 69%|██████▉   | 2766584/4000001 [03:01<01:20, 15271.52it/s]

Part 2: 12543202766584


In [32]:
from numba import jit
import numpy as np

# coords = [[int(i) for i in re.match(pattern=r"[^=]+=(\-?\d+), y=(\-?\d+)[^=]+=(\-?\d+), y=(\-?\d+)", 
#         string=l).groups()] for l in data.split('\n')]
# sensors = [(x1, y1, abs(x1-x2)+abs(y1-y2)) for x1,y1,x2,y2 in coords]

@jit(nopython=True)
def go_fast(S):
    for y in range(0, max_c+1):
        x = 0
        while x <= max_c:
            dm = (S[:,2] - np.abs(x-S[:,0]) - np.abs(y-S[:,1])).max()
            if dm < 0:
                return x*4000000+y
            x += max(1, dm)

print("Part 2:", go_fast(S=np.array(sensors)))

Part 2: 12543202766584


FOUND 12543202766584


12543202766584